## Quickstart: Getting Started with the Snowflake Python API

To start, import the required libraries. We'll import Snowpark, the core Snowflake Python API library, and several classes within the core library that are used to manage Snowflake objects. 

In [1]:
from datetime import timedelta
from typing import List

from snowflake.snowpark import Session
from snowflake.snowpark.functions import col
from snowflake.core import Root,CreateMode
from snowflake.core.database import Database
from snowflake.core.schema import Schema
from snowflake.core.table import Table, TableColumn, PrimaryKey
from snowflake.core.warehouse import Warehouse
from snowflake.core._common import CreateMode
from snowflake.core.task import StoredProcedureCall, Task
from snowflake.core.task.dagv1 import DAGOperation, DAG, DAGTask

Check [Connecting to Snowflake with the Snowflake Python API](https://docs.snowflake.com/en/LIMITEDACCESS/snowflake-python-api/snowflake-python-connecting-snowflake). To get started quickly, create a file `$HOME/.snowflake/config.toml` with the following contents,

```toml
[connections]
[connections.default]
account = "YOUR ACCOUNT NAME"
user = "YOUR ACCOUNT USER"
password = "YOUR ACCOUNT USER PASSWORD"
# optional
# warehouse = "COMPUTE_WH"
# optional
# database = "COMPUTE_WH"
# optional
# schema = "PUBLIC"
```

Next create a `connections_params` dictionary as show below and set the connection to use for the session,

```py
   connection_params = {
	"connection_name": "your connection name",
   }
```

In [29]:
connection_params = {
	"connection_name": "default"
}

Next, create your Snowpark session by passing in your connection parameters.

In [131]:
session = Session.builder.configs(connection_params).create()

Next, instantiate the `Root` class and pass in your session object as an argument. The resulting object – in this case, `root` – will be used to invoke the rest of Snowflake Python API methods and types.

In [132]:
root = Root(session)

Let's start by using `root`  to create a database, schema, and table.

In [133]:
database = root.databases.create(Database(name="PYTHON_API_DB"), mode=CreateMode.or_replace)

There are multiple ways to explore Snowflake objects,

- Navigating to your Snowflake account in the browser and verify the objects that has been created. In this case `PYTHON_API_DB` database.
- Using Snowflake Python API

Let us use the API to list all the database objects and ensure `PYTHON_API_DB` is created.

In [ ]:
all_databases:List[Database] = root.databases.iter()
for db in all_databases:
    print(f"{db.name}")

> TIP:
If you use VSCode, then install [Snowflake plugin](https://marketplace.visualstudio.com/items?itemName=snowflake.snowflake-vsc) to explore all Snowflake objects from within your editor

In [135]:
schema = database.schemas.create(Schema(name="PYTHON_API_SCHEMA"), mode=CreateMode.or_replace)

In [136]:
table = schema.tables.create(
	Table(
		name="PYTHON_API_TABLE", 
	  columns=[
			TableColumn(name="TEMPERATURE", datatype="int", nullable=False), TableColumn(name="LOCATION", datatype="string"),
		],
  ), 
  mode=CreateMode.or_replace)

You should now have a database, schema, and table within your Snowflake account. 

`table` represents a `TableResource` object. To retrieve information about the table, call `.fetch()` on the table you created above and store it an object. 

In [137]:
table_details = table.fetch()

`.fetch()` returns a `TableModel` object, and objects of this type have several methods associated with them that can return more information about that specific Snowflake object. Let's call `.to_dict()` on `python_api_table` to view this in action.

In [ ]:
table_details.to_dict()

As you can see, the dictionary above contains information about the `PYTHON_API_TABLE` table that you created earlier, including information on `columns`, `owner`, `database`, `schema` etc.,

In [138]:
table_details.columns.append(TableColumn(name="elevation",datatype="int",nullable=False,constraints=[PrimaryKey()]))

In the cell above, we define a new column using `TableColumn` and add it to the array in the TableModel that represents the column in the table. To add the column to the table, run the next cell.

In [ ]:
table.create_or_update(table_details)

Navigate to your Snowflake account and confirm that the new column was added to the table.

In [ ]:
table.fetch().to_dict()

Now let's create a warehouse.

In [67]:
warehouses = root.warehouses

The cell above returns the collection of warehouses associated with your session. Let's define a new warehouse.

In [72]:
python_api_wh = Warehouse(
    name="PYTHON_API_WH",
    warehouse_size="SMALL",
    auto_suspend=500,
)

warehouse = warehouses.create(python_api_wh,mode=CreateMode.or_replace)

The cell above defines a new warehouse and creates it in your Snowflake account. Navigate to your account to confirm that the warehouse was created.

Now let's grab some information related to the warehouse.

In [ ]:
warehouse_details = warehouse.fetch()
warehouse_details.to_dict()

We can also iterate through and search the warehouses in your Snowflake account.

In [ ]:
warehouse_list = warehouses.iter(like="PYTHON_API_WH")
result = next(warehouse_list)
result.to_dict()

In the cell above, we search for all warehouses that have a name that matches `PYTHON_API_WH`. In this case, we passed in the exact name of the warehouse we created earlier, but the `like` argument is generally a case-insensitive string that functions as a filter, with support for SQL wildcard characters like `%` and `_`.

Let's now programmatically change the size of the warehouse.

In [76]:
warehouse = root.warehouses.create(Warehouse(
    name="PYTHON_API_WH",
    warehouse_size="LARGE",
    auto_suspend=500,
),mode=CreateMode.or_replace)

Confirm that the warehouse size was updated. You can also navigate to your Snowflake account to confirm the change in warehouse esize.

In [ ]:
warehouse.fetch().size

Finally, delete the warehouse and close your Snowflake connection.

In [78]:
warehouse.delete()

Since we don't yet have the API for manipulating stages, let us create the `TASKS_STAGE` stage using SQL,

In [ ]:
tasks_stage_name = f"{database.name}.{schema.name}.TASKS_STAGE"
create_query = f"CREATE OR REPLACE STAGE {tasks_stage_name}"
session.connection.execute_string(create_query)

## Creating and managing tasks

Create a couple of functions and manage them as `Task`s:

In [118]:
def trunc(session: Session, from_table: str, to_table: str, count: int) -> str:
  session.table(from_table).limit(count).write.save_as_table(to_table)
  return "Truncated table successfully created!"

def filter_by_shipmode(session: Session, mode: str) -> str:
  session.table("snowflake_sample_data.tpch_sf100.lineitem").filter(col("L_SHIPMODE") == mode).limit(10).write.save_as_table("filter_table")
  return "Filter table successfully created!"

In [120]:

task1 = Task(
    "task_python_api_trunc",
    definition=StoredProcedureCall(trunc, stage_location=f"@{tasks_stage_name}", packages=["snowflake-snowpark-python"]),
    warehouse="COMPUTE_WH",
    schedule=timedelta(minutes=1)
)

task2 = Task(
    "task_python_api_filter",
    definition=StoredProcedureCall(filter_by_shipmode, stage_location=f"@{tasks_stage_name}", packages=["snowflake-snowpark-python"]),
    warehouse="COMPUTE_WH"
)

Create the tasks in the specified database by adding them to the list of tasks associated with the database and schema referenced:

In [ ]:
# create the task into the Snowflake database
tasks = schema.tasks

trunc_task = tasks.create(task1, mode=CreateMode.or_replace)
# should be fully qualified name
task2.predecessors = [f"{trunc_task.database.name}.{trunc_task.schema.name}.{trunc_task.name}"]
filter_task = tasks.create(task2, mode=CreateMode.or_replace)

Tasks are suspended by default when created. Start the `trunc_task` by calling `.resume()` on it:

In [122]:
trunc_task.resume()

You can retrieve more information about tasks by iterating through them and printing out their name and state. Run the following cell to observe the status of each task:

In [ ]:
taskiter = tasks.iter()
for t in taskiter:
    print("Name: ", t.name, "| State: ", t.state)

Navigate to your Snowflake account and confirm that `trunc_task` was indeed started. After confirming, suspend it by running the following cell:

In [124]:
trunc_task.suspend()

Finally, delete both tasks:

In [125]:
trunc_task.delete()
filter_task.delete()

## Creating and managing DAGs

The following cell defines a DAG using the `DAG` constructor, and creates tasks within the DAG by calling the `DAGTask` constructor. Note that the arguments passed in to this constructor are the same ones that were passed in to the `Task` constructor in an earlier cell. We also use `>>` to specify a root task. Finally, we create the DAG in our referenced schema and deploy it to Snowflake:

In [ ]:
dag_name = "python_api_dag"
dag = DAG(dag_name, schedule=timedelta(days=1))
with dag:
    dag_task1 = DAGTask(
        "task_python_api_trunc",
        StoredProcedureCall(
            trunc,
            stage_location=f"@{tasks_stage_name}",
            packages=["snowflake-snowpark-python"]),
        warehouse="COMPUTE_WH",
    )
    dag_task2 = DAGTask(
        "task_python_api_filter",
        StoredProcedureCall(
            filter_by_shipmode,
            stage_location=f"@{tasks_stage_name}",
            packages=["snowflake-snowpark-python"]),
        warehouse="COMPUTE_WH",
    )
    dag_task1 >> dag_task2
dag_op = DAGOperation(schema)
dag_op.deploy(dag, mode=CreateMode.or_replace)

To run the DAG, run the root task:

In [127]:
dag_op.run(dag)  # Run the root task.

Navigate to your Snowflake account and confirm that the root task in the DAG was started. Once confirmed, delete the DAG:

In [128]:
dag_op.delete(dag)

Clean up all the objects that was created with this quickstart and close the session

In [140]:
database.delete()
session.close()

Congratulations! In this notebook, you learned the fundamentals for managing Snowflake objects, tasks, and DAGs using the Snowflake Python API.

For more information, see the following resources:

* [Snowflake Documentation: Snowflake Python API](https://docs.snowflake.com/en/LIMITEDACCESS/snowflake-python-api/snowflake-python-overview)

* [Snowflake Python API Reference Documentation](https://docs.snowflake.com/en/LIMITEDACCESS/snowflake-python-api/reference/0.1.0/index.html)